## _calculate_flux : calculates air-sea CO2 flux

This notebook implements Wannikhof (1992) to calculate the air-sea CO2 flux:
$F_{CO2}=k_w · S_{CO2} ·(1−f_{ice})·(pCO2_{atm}−pCO2_{ocn})$

### Variables
- Gas transfer velocity : $k_w$ $\frac{cm}{hr}$ 
- CO2 solubility ; $S_{CO2}$= $\frac{mol}{kg\ \mu atm}$
- partial pressure CO2 in moist air : $pCO2_{atm}$ = $\mu atm$
- partial pressure CO2 in ocean: $pCO2_{ocn}$ = $\mu atm$
- CO2 flux across air-sea interface : $F_{CO2}$ = $\frac{cm\ mol}{hr\ kg}$

### References 
- Wanninkhof (1992)
- Sweeney et al. (2007)
- Weiss (1974)
- Dickson et al. (2007)

In [1]:
%matplotlib inline
import xarray as xr
import xarray.ufuncs as xu
import numpy as np
import cmocean as cm
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import scipy.io
import scipy
from scipy.stats import stats

from distributed import Client
from processing import processing as pr
from skill_metrics import skill_metrics as sk

In [2]:
%run _define_model_class.ipynb
%run _define_plotting_class.ipynb

# air-sea CO2 flux

In [3]:
def wanninkhof92(T=None, 
                 S=None, 
                 P=None,
                 u_mean=None, 
                 u_var=None, 
                 xCO2=None,
                 pCO2_sw = None,
                 iceFrac = None,
                 scale_factor=0.27):
    '''
    Calculate air-sea CO2 flux following Wanninkhof 1992
    
    Inputs
    ============
    T = Temperature [degC]
    S = Salinity  [parts per thousand]
    P = Atmospheric pressure at 10m [atm]
    u_mean = mean wind speed [m/s]
    u_var = variance of wind speed m/s averaged monthly [m^2/s^2]
    xcO2 = atmoapheric mixing ratio of CO2 [ppm]
    pCO2_sw = CO2 partial pressure of seawter [uatm]
    scale_factor = gas transfer scale factor (default=0.27)
    
    Output
    ============
    F_co2 = air-sea co2 flux [molC/m2/yr]
    
    References
    ============
    Weiss (1974) [for solubility of CO2]
        Carbon dioxide in water and seawater: the solubility of a non-ideal gas
    Weiss and Price (1980) [for saturation vapor pressure]
        Nitrous oxide solubility in water and seawater     
    Dickson et al (2007) [for saturation vapor pressure]
        Guide to best practices for Ocean CO2 measurements, Chapter 5 section 3.2
    Wanninkhof (1992) [for gas transfer and Schmidt number]
        Relationship between wind speed and gas exchange over the ocean   
    Sweeney et al. (2007) [for gas transfer scale factor]
        Constraining global air‐sea gas exchange for CO2 with recent bomb 14C measurements
    Sarmiento and Gruber (2007) [for overview and discussion of CO2 flux]
        Ocean biogeochmiecal dynsmics, Ch3 see panel 3.2.1
    
    Notes
    ============
    this code is optimized to work with Xarray
    
    ## Notes on partial pressure moist air
    P_h20/P = saturation vapor pressure (Weiss and Price 1980)
    Water vapor is generally assumed to be at saturation in the vicinity of the air-sea interfae.
        p_moist = X_dry (P - P_h20)
                = x_dry * P (1-P_h20/P)
                = P_dry (1-P_h20/P)

    ## Notes on U2
    Need to add wind speed variance if using Wanninkhof (1992)
    If we don't, then we are underestimating the gas-transfer velocity
    U2 = (U_mean)^2 + (U_prime)^2, (U_prime)^2 is the standard deviation
    See Sarmiento and Gruber (2007) for a discussion of this
    
    ## Test values
    # T = 298.15
    # S = 35
    
    '''
    
    ### ================================================
    ### 0. Conversions
    ### ================================================
    ### Convert from Celsius to kelvin
    T_kelvin = T + 273.15
    
    
    ### ================================================
    ### 1. partial pressure of CO2 in moist air
    ###    Units : uatm
    ### ================================================    
    ### Weiss and Price (1980) formula
    # P_sat_vapor = xu.exp(24.4543 - 67.4509*(100/T_kelvin) - 4.8489*xu.log(T_kelvin/100) - 0.000544*S)
    
    
    ### Following Dickson et al (2007) Chapter 5 section 3.2
    a1 = -7.85951783
    a2 = 1.84408259
    a3 = -11.7866497
    a4 = 22.6807411
    a5 = -15.9618719
    a6 = 1.80122502

    Tc = 647.096
    pc = (22.064 * 10**6) / 101325
    g = (1-(T_kelvin/Tc))

    ### pure water saturation vapor pressure, Wagner and Pruß, 2002
    p_h20 = pc * xu.exp((Tc/T_kelvin) * (a1*g + 
                          a2*g**(1.5) + 
                          a3*g**(3) + 
                          a4*g**(3.5) + 
                          a5*g**(4) + 
                          a6*g**(7.5)))


    ### total molality 
    MT = (31.998*S) / (1000 - 1.005*S)

    ### osmotic coefficient at 25C by Millero (1974)
    b1 = 0.90799 
    b2 = - 0.08992
    b3 = 0.18458
    b4 = -0.07395
    b5 = -0.00221
    phi = b1 + b2*(0.5*MT) + b3*(0.5*MT)**2 + b4*(0.5*MT)**3 + b5*(0.5*MT)**4

    ### vapor pressure above sea-water, Dickson et al. (2007), Chapter 5, section 3.2
    P_sat_vapor = p_h20 * xu.exp(-0.018*phi*MT)

    ### Partial pressure of CO2 in moist air, Dickson et al. (2007), SOP 4 section 8.3
    P_moist = (P * xCO2) * (1 - P_sat_vapor)
    
    
    ### ================================================
    ### 2. Solubility of CO2
    ###    Notes : T in degC, S in PSU, 
    ###            1.0E-6 converts to correct to muatm
    ###    Units : to mol.kg^{-1}uatm^{-1}
    ###    Reference : Weiss (1974)
    ### ================================================  
    S_co2 = xu.exp( 9345.17 / T_kelvin - 60.2409 + \
                     23.3585 * xu.log( T_kelvin / 100 ) + \
                     S * ( 0.023517 - 0.00023656 * T_kelvin + \
                          0.0047036 * ( T_kelvin / 100 )**2 )) *1.0E-6
    
    
    ### ================================================
    ### 3. Gas transfer velocity
    ###    Units : cm/hr
    ###    Reference : Wanninkhof (1992)
    ###                Sweeney et al. (2007)
    ###                per.comm with P. Landschutzer Feb. 19 2019
    ### ================================================  
    # Dimensionless Schmidt number (Sc)
    # References Wanninkhof 1992
    A = 2073.1 
    B = 125.62 
    C = 3.6276 
    D = 0.043219
    
    # Schmidt number 
    # units : dimensionless
    Sc = A - B*T + C*T**2 - D*T**3
    
    # Gas transfer velocity
    # References :  Wanninkhof 1992, 
    # Sweeney et al. 2007 scale factor
    k_w = scale_factor * (Sc/660)**(-0.5) * (u_mean**2 + u_var)

    
    ### ================================================
    ### 4. air-sea CO2 exchange
    ###    Units : mol/m2/yr
    ###    Reference : Wanninkhof (1992)
    ### ================================================  
    # Convert from cm*mol/hr/kg to mol/m2/yr
    conversion = (1000/100)*365*24
    
    # Air-sea CO2 flux 
    F_co2 = k_w * S_co2 * (1 - iceFrac) * (P_moist - pCO2_sw) * conversion
    
    return F_co2

# calculate flux

In [4]:
def calculate_flux(model=None, member=None):
    dir_raw = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/raw'
    dir_clean = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean'
    
    ### ================================================
    ### Get file path
    ### ================================================
    if model.upper()=='CESM':
        mem = f'{member:0>3}'
    if model.upper()=='GFDL':
        mem = f'{member:0>2}'
    if model.upper()=='MPI':
        mem = f'{member:0>3}'
    if model.upper()=='CANESM2':
        mem = f'{member}'

    ### Load auxillary data
    ds_SST     = read_model2(model=model, member=mem, variable='SST')
    ds_SSS     = read_model2(model=model, member=mem, variable='SSS')
    ds_iceFrac = read_model2(model=model, member=mem, variable='iceFrac')
    ds_pATM    = read_model2(model=model, member=mem, variable='pATM')
    ds_XCO2    = read_model2(model=model, member=mem, variable='XCO2')
    ds_U10     = read_model2(model=model, member=mem, variable='U10')
    ds_pCO2    = read_model2(model=model, member=mem, variable='pCO2')
    ds_pCO2_SOMFFN = read_somffn(model=model, member=mem)
    
    ### Wind speed variance 
    ds_U_var = xr.open_dataset(f'{dir_clean}/ERA_interim/ERAinterim_1x1_u10-var_1982-2016.nc', 
                               decode_times=False)

    ###======================================
    ### Put data into xarray dataset
    ###======================================
    ds = xr.Dataset(
        {
        'SST':(['time','lat','lon'], ds_SST['SST'] ),
        'SSS':(['time','lat','lon'], ds_SSS['SSS']),
        'iceFrac':(['time','lat','lon'], ds_iceFrac['iceFrac'] ),
        'U10':(['time','lat','lon'], ds_U10['U10']),
        'U10_var':(['time','lat','lon'], ds_U_var['u10']),
        'Patm':(['time','lat','lon'], ds_pATM['pATM'] ),
        'xCO2':(['time'], ds_XCO2['XCO2']),
        'pCO2_member':(['time','lat','lon'], ds_pCO2['pCO2']),
        'pCO2_somffn':(['time','lat','lon'], ds_pCO2_SOMFFN['pco2']),
        },

        coords={
        'lat': (['lat'], ds_pCO2_SOMFFN['lat']),
        'lon': (['lon'], ds_pCO2_SOMFFN['lon']),
        'time': (['time'], ds_pCO2_SOMFFN['time'])
        })

    F_somffn = wanninkhof92(T=ds['SST'], 
                 S=ds['SSS'], 
                 P=ds['Patm'],
                 u_mean=ds['U10'], 
                 u_var=ds['U10_var'], 
                 xCO2=ds['xCO2'],
                 pCO2_sw = ds['pCO2_somffn'],
                 iceFrac = ds['iceFrac'],
                 scale_factor=0.27*1.014)
        
    F_member = wanninkhof92(T=ds['SST'], 
                 S=ds['SSS'], 
                 P=ds['Patm'],
                 u_mean=ds['U10'], 
                 u_var=ds['U10_var'], 
                 xCO2=ds['xCO2'],
                 pCO2_sw = ds['pCO2_member'],
                 iceFrac = ds['iceFrac'],
                 scale_factor=0.27*1.014)


    ds_out = xr.Dataset(
        {
        'F_member':(['time','lat','lon'], F_member), 
        'F_somffn':(['time','lat','lon'], F_somffn),       
        },

        coords={
        'lat': (['lat'], ds['lat']),
        'lon': (['lon'], ds['lon']),
        'time': (['time'], ds['time'])
        })
        
    ds_out.to_netcdf(f'{dir_clean}/CO2_flux/CO2_flux_{model}{mem}_SOMFFN.nc')


# Start Dask client

In [5]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:33514 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 48 Cores: 48 Memory: 269.92 GB


In [8]:
client.restart()

Client Scheduler: tcp://127.0.0.1:33514 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 48 Cores: 48 Memory: 269.92 GB


In [9]:
###=================================================================
### LE_model: ['CESM', 'GFDL', 'MPI', CanESM2]
### model_or_recon: ['MODEL', 'SOMFFN', 'UEASI']
###=================================================================
for LE_model in ['CanESM2']:
    ###======================================
    ### Get members from model
    ###======================================
    if LE_model=='CESM':
        members=[1,2,9,10,11,12,13,14,15,16,17,18,20,21,23,24,25,30,31,34,35,101,102,103,104]
        n = len(members)
        print(f'Loading {n} members from {LE_model}')

    if LE_model=='GFDL':
        members=[1,2,3,4,5,6,8,9,10,11,12,13,14,16,17,18,19,20,22,23,26,27,28,29,30]
        n = len(members)
        print(f'Loading {n} members from {LE_model}')

    if LE_model=='MPI':
        members=[6,9,14,20,22,24,25,27,38,43,45,46,57,60,64,70,75,77,78,80,81,83,91,95,98]
        n = len(members)
        print(f'Loading {n} members from {LE_model}')

    if LE_model=='CanESM2':
        members=['r1r10', 'r1r6', 'r1r7', 'r1r9', 'r2r1', 'r2r2', 'r2r8', 'r3r1', 'r3r2', 'r3r4',
                'r3r6', 'r3r7', 'r3r9', 'r4r1', 'r4r3', 'r4r5', 'r4r6', 'r4r7', 'r4r8', 'r5r1', 
                'r5r10', 'r5r2', 'r5r4', 'r5r5', 'r5r9']
        n = len(members)
        print(f'Loading {n} members from {LE_model}')

    ###======================================
    ### Load data
    ### client.map    --> These results live on distributed workers.
    ### client.submit --> We can submit tasks on futures. 
    ###                   The function will go to the machine where 
    ###                   the futures are stored and run on the result 
    ###                   once it has completed.
    ###======================================
    print(f'Calculating flux...')
    fut = client.map(calculate_flux, tuple(np.repeat(LE_model, n)), members)
    client.gather(fut)
    
print('Complete!!')

Loading 25 members from CanESM2
Calculating flux...
Complete!!


In [10]:
#del fut